### Section 0: Setup
If running this in Google Colab, make sure that you are connected to a GPU instance and run the install script below. It should (hopefully) take about 2-5mins to execute.

In [ ]:
import subprocess
import os

# Check if GPU exists
try:
    subprocess.check_output('nvidia-smi')
    print("GPU is enabled.")
    # Check if running in Google Colab
    if 'COLAB_GPU' in os.environ:
      # taken from https://colab.research.google.com/github/nerfstudio-project/nerfstudio/blob/main/colab/demo.ipynb
      %cd /content/
      !pip install --upgrade pip
      !pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 --extra-index-url https://download.pytorch.org/whl/cu118

      # Installing TinyCuda
      %cd /content/
      !gdown "https://drive.google.com/u/1/uc?id=1-7x7qQfB7bIw2zV4Lr6-yhvMpjXC84Q5&confirm=t"
      !pip install tinycudann-1.7-cp310-cp310-linux_x86_64.whl
      !pip install commentjson
      !pip install pytorch_lightning
      # broken cuda version
      !pip uninstall -y torchaudio
    else:
      print("COLAB_GPU not detected")
except FileNotFoundError as e:
    print("GPU is not enabled in this notebook.")
    print("Please select 'Runtime -> Change runtime type' and set the hardware accelerator to GPU.")



In [ ]:
# Clone repo (Colab only)
if 'COLAB_GPU' in os.environ:
    !git clone https://github.com/princeton-computational-imaging/SoaP/
    %cd /content/SoaP/

In [ ]:
!wget https://soap.cs.princeton.edu/shade_map.npy -P data/
!wget https://soap.cs.princeton.edu/demo.zip -P data/
!unzip data/demo.zip -d data/

### Section 1: (Optional) What is a `frame_bundle.npz`?


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from utils import utils

Load data from disk:

In [ ]:
bundle_path = "data/demo/dragon/compressed_frame_bundle.npz"
# convert to dictionary - important, by default npz load as a namespace which can have odd behaviour
bundle = dict(np.load(bundle_path, allow_pickle=True))
# remove extra dimensions
utils.de_item(bundle)

Our bundle contains four sets of data:  
1. `motion` : device motion data including rotation, gravity, and acceleration  
2. `raw_[x]` : Bayer RAW frames enumerated from `0` to `num_raw_frames - 1`, with associated metadata  
3. `rgb_[x]` : Processed Apple RGB frames enumerated from `0` to `num_rgb_frames - 1`, with associated metadata  
4. `depth_[x]` : Apple depth maps enumerated from `0` to `num_depth_frames - 1`, with associated metadata  

Lets take a closer look at this data:

In [ ]:
bundle["motion"].keys()

The motion data `motion` contains:  
1. `frame_count` : what frame was being recorded when the associated motion data was recorded. There can be multiple motion values for the same frame as the frequency of the accelerometer/gyroscope (100Hz) is higher than the framerate we're recording at (21fps).
2. `timestamp` : absolute device time at which measurements were recorded
3. `quaternion` : device relative rotation expressed in quaternion format
4. `rotation_rate` : velocity of device rotation expressed in roll-pitch-yaw
5. `roll_pitch_yaw` : device relative rotation expressed in roll-pitch-yaw
6. `acceleration` : device relative acceleration (with gravity removed) expressed in x-y-z
7. `gravity` : acceleration due to gravity expressed in x-y-z
8. `num_motion_frames` : number of recorded measurements

As an example lets plot the device roll over time:

In [ ]:
roll_pitch_yaw =  bundle["motion"]["roll_pitch_yaw"] # [3,N]
timestamp =  bundle["motion"]["timestamp"] # [N]
roll = roll_pitch_yaw[:,0]
pitch = roll_pitch_yaw[:,1]
yaw = roll_pitch_yaw[:,2]

plt.plot(timestamp, roll)
plt.ylabel("Roll [Rad]")
plt.xlabel("Device Time [s]")
plt.show()

RAW image data:

In [ ]:
frame = 0 # change this to view other frames
raw = bundle[f"raw_{frame}"]
rgb = bundle[f"rgb_{frame}"]
depth = bundle[f"depth_{frame}"]

In [ ]:
print(raw.keys())
print("height:", raw['height'], "width:", raw['width'])

Each `raw` frame consists of:
1. `frame_count` : frame number, ranges from 0 - `num_raw_frames`
2. `timestamp` : absolute device time at which frame was recorded
3. `height, width` : frame dimensions (**WARNING**: these may not match the expected orientation of the frame, i.e. if you are recording with the phone vertical or horizontal, the `width` does not change and always refers to the long side of the capture)
4. `ISO`, `exposure_time`, `aperture` : camera ISO, exposure time (seconds), and f-stop used to capture the image
5. `brightnesss` : the estimated 'brightness' of the scene, honestly not sure what this is (pls message me if you know)
6. `shutter_speed` : inverse of `exposure_time`
7. `black_level`, `white_level`: min and max real RAW values
8. `raw`, 4032 x 3024 single channel, 14-bit mosaiced bayer CFA frame

Lets look at the RAW image data:

In [ ]:
raw_img = raw["raw"]

# use simple demosaicing the fill gap values (see paper supplemental)
raw_demosaiced = utils.raw_to_rgb(torch.tensor(raw_img[None,None].astype(np.int32)))[0].permute(1,2,0)
raw_demosaiced = raw_demosaiced/raw_demosaiced.max()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
axes[0].imshow(raw_img, cmap="gray")
axes[0].set_title(f"Frame {frame} Mosaiced Raw")
im = axes[1].imshow(raw_demosaiced)
axes[1].set_title(f"Frame {frame} De-Mosaiced Raw")

fig.subplots_adjust(right=0.7)
plt.show()

If we zoom into a small patch of the above mosaiced RAW we can see the Bayer CFA pattern:

In [ ]:
plt.imshow(raw_img[:8,:8], cmap="gray")
plt.show()

Applying the shade map to this data we see how it corrects for the vignetting on the edges of the scene:

In [ ]:
shade_map = np.load("data/shade_map.npy")
raw_img_deshade = raw["raw"] * shade_map

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))
axes[0].imshow(shade_map, cmap="gray")
axes[0].set_title(f"Shade Map")
im = axes[1].imshow(raw_img_deshade, cmap="gray")
axes[1].set_title(f"Frame {frame} Mosaiced Raw + Shade Map")

fig.subplots_adjust(right=0.7)
plt.show()

Processed RGB and depth data:

In [ ]:
print(rgb.keys())
print("height:", rgb['height'], "width:", rgb['width'])

Each `rgb` frame contains:
1. `frame_count`, `timestamp`, `height`, `width` : see `raw` documentation
2. `intrinsics`: 3x3 camera intrinsics, see: [documentation](https://developer.apple.com/documentation/avfoundation/avcameracalibrationdata/2881135-intrinsicmatrix)
3. `rgb`, 1920 x 1440 3 channel, 8-bit processed RGB frame

In [ ]:
print(depth.keys())
print("height:", depth['height'], "width:", depth['width'])

Each `depth` frame contains:
1. `frame_count`, `timestamp`, `height`, `width` : see `raw` documentation
2. `intrinsic_height`, `intrinsic_width`, `intrinsics` : 3x3 camera intrinsics, with associated frame height and width
3. `lens_distortion` : look-up table for radial distortion correction, see: [documentation](https://developer.apple.com/documentation/avfoundation/avcameracalibrationdata/2881129-lensdistortionlookuptable)
4. `lens_undistortion` : inverse of `lens_distortion`
5. `depth_accuracy` : [accuracy of depth measurements](https://developer.apple.com/documentation/avfoundation/avdepthdata/accuracy), depends on iPhone/iOS version. `1` -> metric depth, `0` -> relative depth
6. `depth`, 320 x 240 inverse depth map from monocular cues + LiDAR measurements

Here's a preview of what the RGB and iPhone depth data look like:

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 8))
axes[0].imshow(rgb['rgb'])
axes[0].set_title("Frame {0} Image".format(frame))
im = axes[1].imshow(depth['depth'], cmap='RdYlBu')
axes[1].set_title("Frame {0} iPhone Depth".format(frame))

fig.subplots_adjust(right=0.82)
cbar_ax = fig.add_axes([0.85, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label='Depth [m]')
plt.show()

print()
print("Camera Info at Frame {0}: \n".format(frame))
print("Timestamp:", rgb['timestamp'], "\n")
print("Camera Intrinsics: \n", rgb['intrinsics'])

### Section 2: Training on a `frame_bundle.npz`
This section will cover how to fit our model to an input RAW frame_bundle.npz, monitor the model's training, and plot its outputs.


In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from train import *
from utils import utils

Lets begin by taking a look at the images in our `compressed_frame_bundle.npz` (this is a sub-sampled `frame_bundle.npz` with 9 images instead of 42 to speed up training/download time)

In [ ]:
bundle = dict(np.load("data/demo/dragon/compressed_frame_bundle.npz", allow_pickle=True))
utils.de_item(bundle)

# plot the first 5 images, downsample 2x for speed
fig, ax = plt.subplots(1,5, figsize=(19.5,5))
for i in range(5):
    ax[i].imshow(bundle[f"rgb_{i}"]["rgb"][::2,::2])
    ax[i].set_title(f"Image {i}")

# remove ticks
for a in ax:
    a.tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
# adjust spacing
plt.subplots_adjust(wspace=0.0)
plt.show()

While they barely appear to change, there's actually still more than enough parallax here to recover meaningful depth.  

We begin by launching tensorboard so we can see our training progress:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

Next we run `train.py`. On an RTX 4090 this should train in a couple minutes, on Colab this will be quite a bit slower.  

You can refresh the tensorboard window above to watch the training progress.

In [ ]:
# only run to 30 epochs to save time, remove the flag to run for default 100 epochs
!python3 train.py --name dragon-test --bundle_path data/demo/dragon/compressed_frame_bundle.npz --max_epochs 30

To view our reconstruction we load the model from disk:

In [ ]:
model = BundleMLP.load_from_checkpoint("checkpoints/dragon-test/last.ckpt", device="cuda")

In [ ]:
# move model components to GPU
model = model.eval()
model = model.to('cuda')
model.rgb_volume = model.rgb_volume.to('cuda')
model.processed_rgb_volume = model.processed_rgb_volume.to('cuda')
model.model_rotation = model.model_rotation.to('cuda')
model.model_translation = model.model_translation.to('cuda')
model.reference_intrinsics = model.reference_intrinsics.to('cuda')
model.model_rotation.reference_rotation = model.model_rotation.reference_rotation.to('cuda')

# use all encoding levels for inference
model.mask = torch.ones_like(model.mask)

And use `model.generate_outputs` to generate the outputs:

In [ ]:
rgb, rgb_raw, rgb_processed, depth, depth_img = model.generate_outputs(frame=0, height=1920, width=1440, u_lims=[0.025,0.975], v_lims=[0.025,0.975])

Outputs:
1. `rgb` : color values I(u,v) output by implicit image model
2. `rgb_raw` : corresponding sampled values from bayer RAW volume
3. `rgb_processed` : corresponding sampled values from processed RGB volume
4. `depth` : depth values D(u,v) from shakes-on-a-plane implicit depth model
5. `depth_img` : same as `depth` but with colormap applied for tensorboard visualization

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 8))
axes[0].imshow((rgb.permute(1,2,0).cpu()).clip(0,1)) # increase brightness
axes[0].set_title("Reconstructed Image I(u,v)")
axes[1].imshow(rgb_processed.permute(1,2,0).cpu())
axes[1].set_title("Processed RGB")
axes[2].imshow(depth.cpu(), cmap="RdYlBu")
axes[2].set_title("Reconstructed Depth D(u,v)")
plt.show()

### Section 3: Training on PNGs
This section is almost identical to the previous one, except we will learn how to convert a stack of `PNGs` into a `frame_bundle.npz` before fitting our model to it.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import utils.utils as utils
from glob import glob
from train import *

You can replace the code below with any filetype (e.g., load an MP4 with OpenCV), as long as `imgs` is a `NxHxWxC` array, where N is the number of frames.

In [ ]:
imgs = sorted(glob("data/demo/dragon-rgb/*.png")) # change file extension to match your filetypes
imgs = np.array([plt.imread(img)[:,:,:3] for img in imgs]) # remove alpha channel and load

print("Number of images: ", len(imgs))
# plot first image, last image
fig, ax = plt.subplots(1,2, figsize=(10,5))
ax[0].imshow(imgs[0])
ax[0].set_title("Image 0")
ax[1].imshow(imgs[-1])
ax[1].set_title(f"Image {len(imgs)-1}")
plt.show()

For our projective camera model to work we'll need to supply it with [camera intrinsics](https://en.wikipedia.org/wiki/Camera_matrix). Here we'll assume we don't have and calibrated intrinsics and will have to create our own.

We'll set the camera centers `cx` and `cy` to be the center of the image:

In [ ]:
cy = imgs.shape[1] // 2 # set centers to the middle of the image
cx = imgs.shape[2] // 2
print("Center y: ", cy, "\nCenter x: ", cx)

If we don't know the focal length of the camera, we can use a best guess of its FOV (around 70 degrees for a standard phone camera) to calculate it:

In [ ]:
focal = min(cx, cy)/np.tan(70 * (np.pi/180/2)) # 70 degree field of view
print("Focal length (pixels): ", focal)

In [ ]:
intrinsics = np.array([[focal, 0, 0],
                       [0, focal, 0],
                       [cx, cy, 1]])

These and the images are all we need to make our custom frame bundle, which we save to the same folder as the input data:

In [ ]:
rgb_bundle = {}
for i in range(len(imgs)):
    rgb = {"rgb": imgs[i], "intrinsics": intrinsics, "height": imgs.shape[2], "width": imgs.shape[1]}
    rgb_bundle[f'rgb_{i}'] = rgb
rgb_bundle['num_rgb_frames'] = len(imgs)
rgb_bundle['num_raw_frames'] = 0
rgb_bundle['num_depth_frames'] = 0
rgb_bundle['motion'] = None
np.savez('data/demo/dragon-rgb/frame_bundle.npz', **rgb_bundle)

Now we can train our model as before:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs

However we now have to add flags `--no_device_rotations`, `--no_phone_depth`, and `--no_raw` to let the training code know that we're only passing in RGB data and nothing else.

In [ ]:
# only run to 30 epochs to save time, remove the flag to run for default 100 epochs
!python3 train.py --name dragon-rgb-test --bundle_path data/demo/dragon-rgb/frame_bundle.npz --max_epochs 30 --no_device_rotations --no_phone_depth --no_raw

To view our reconstruction we load the model from disk:

In [ ]:
model = BundleMLP.load_from_checkpoint("checkpoints/dragon-rgb-test/last.ckpt", device="cuda")

In [ ]:
# move model components to GPU
model = model.eval()
model = model.to('cuda')
model.rgb_volume = model.rgb_volume.to('cuda')
model.processed_rgb_volume = model.processed_rgb_volume.to('cuda')
model.model_rotation = model.model_rotation.to('cuda')
model.model_translation = model.model_translation.to('cuda')
model.reference_intrinsics = model.reference_intrinsics.to('cuda')
# model.model_rotation.reference_rotation = model.model_rotation.reference_rotation.to('cuda') # doesnt exist

# use all encoding levels for inference
model.mask = torch.ones_like(model.mask)

And use `model.generate_outputs` to generate the outputs:

In [ ]:
rgb, rgb_raw, rgb_processed, depth, depth_img = model.generate_outputs(frame=0, height=1920, width=1440, u_lims=[0.025,0.975], v_lims=[0.025,0.975])

Outputs:
1. `rgb` : color values I(u,v) output by implicit image model
2. `rgb_raw` : corresponding sampled values from bayer RAW volume
3. `rgb_processed` : corresponding sampled values from processed RGB volume
4. `depth` : depth values D(u,v) from shakes-on-a-plane implicit depth model
5. `depth_img` : same as `depth` but with colormap applied for tensorboard visualization

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(14, 8))
axes[0].imshow((rgb.permute(1,2,0).cpu()).clip(0,1)) # increase brightness
axes[0].set_title("Reconstructed Image I(u,v)")
axes[1].imshow(rgb_processed.permute(1,2,0).cpu())
axes[1].set_title("Processed RGB")
axes[2].imshow(depth.cpu(), cmap="RdYlBu")
axes[2].set_title("Reconstructed Depth D(u,v)")
plt.show()